In [413]:
import pandas as pd

df = pd.read_csv('amlo_clasify_chatpgt3.csv')
#df = df.head(50)
#df.reset_index(inplace=True)
df

,Texto,Clasificacion,new_classification,classification_spanish
0,Buenos días y feliz año. Deseamos que nos vay...,saludos,Support,apoyo
1,Vamos a iniciar la semana como lo hacemos siem...,NaN,Health,salud
2,También vamos a informar este primer día hábil...,apoyo,Support,apoyo
3,Y ahora se va a informar porque se está optand...,apoyo,Support,apoyo
4,En esta etapa se combinan los bancos particula...,apoyo,Support,apoyo
...,...,...,...,...
21760,Y el presidente Modi es el presidente mejor ca...,NaN,NaN,NaN
21761,"¿La tienes ahí? Ponla, sí, porque no le… Por e...",NaN,NaN,NaN
21762,"Una vez, una vez le llegué por medio punto, o ...",NaN,NaN,NaN
21763,"Y hay fotos, ¿no? En la India usan esto. Pero,...",NaN,NaN,NaN


In [414]:
import os
with open(r'C:\Users\claud\Documents\Descargas\open_ai_key.txt') as f:
    os.environ["OPENAI_API_KEY"] = f.readline()

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name= "gpt-3.5-turbo", request_timeout =30, verbose = True, max_retries = 5)


from langchain.schema.messages import HumanMessage, SystemMessage
prompt='''

You are a virtual assistant, who only knows how to classify texts in Spanish and responds only in Spanish, he cannot respond in English.

The classifications that you should take into account are:
Security: talk about security in Mexico
History: facts from the history of Mexico or the world, no comments.
Economy: Refers to the economic situation in Mexico.
Foreign: Talk about how other countries get involved with Mexico or how Mexico impacts abroad, also about migration.
Opinion: People's personal opinion on any topic, mostly opinions about history in the actual society.
Health: Health situation in Mexico.
Support: social and financial support to communities, older adults, indigenous people, etc.
Corruption: About the wrongdoings of other groups or organizations.
Opposition: Opposed to certain ideas or thoughts, even opposition or badmouthing about a person, organization or Politics People.
Construction: Infrastructure built or purchased for the betterment of Mexico.


The answer should only be one word in Spanish for each sentence, not English.

Given you are the virtual assistant classify the following 5 sentences enclosed by quotation marks and separated with commas.

'''

In [415]:
result = []
indice = 0
result2 = []

#decena_de_enunciados = ""
for i in range(0, len(df), 10):
    result.append('","'.join(df['Texto'].iloc[i:i+10]))
    result[indice] = '"' + result[indice]
    result[indice] = result[indice] + '"'
    indice+=1
#for i in range(0, len(df) - 5, 10):
#    decena_de_enunciados = ""
#    for j in range(10):
#        decena_de_enunciados = decena_de_enunciados + f"{j+1}. "
#        decena_de_enunciados = decena_de_enunciados + df["Texto"].iloc[i+j] + '\n'
#    result2.append(decena_de_enunciados)

    

In [416]:
inicio = df["new_classification"].index.get_loc(df['new_classification'].last_valid_index())
inicio = (inicio + 1) //10
print(inicio)
fin = len(df) //10
print(fin)

2176
2176


In [417]:

import re


In [418]:
df

,Texto,Clasificacion,new_classification,classification_spanish
0,Buenos días y feliz año. Deseamos que nos vay...,saludos,Support,apoyo
1,Vamos a iniciar la semana como lo hacemos siem...,NaN,Health,salud
2,También vamos a informar este primer día hábil...,apoyo,Support,apoyo
3,Y ahora se va a informar porque se está optand...,apoyo,Support,apoyo
4,En esta etapa se combinan los bancos particula...,apoyo,Support,apoyo
...,...,...,...,...
21760,Y el presidente Modi es el presidente mejor ca...,NaN,NaN,NaN
21761,"¿La tienes ahí? Ponla, sí, porque no le… Por e...",NaN,NaN,NaN
21762,"Una vez, una vez le llegué por medio punto, o ...",NaN,NaN,NaN
21763,"Y hay fotos, ¿no? En la India usan esto. Pero,...",NaN,NaN,NaN


In [419]:
def clasificate_csv(text):

        response = llm.invoke(
                [SystemMessage(content=prompt), HumanMessage(content=text)]
            )
        return response.content

def classify_df():    
    
    inicio = df["new_classification"].index.get_loc(df['new_classification'].last_valid_index())
    inicio = (inicio + 1) //10 
    fin = len(df) //10
    query_is_not_ten = True
    for i in range(inicio,fin):
        query_is_not_ten = True
        while query_is_not_ten:
            resultado_de_la_clasificacion = clasificate_csv(result[i])
            if len(resultado_de_la_clasificacion.split(", ")) == 10:
                query_is_not_ten = False
            else:
                query_is_not_ten = True
            print(f"Repeticion {i}")
        
        df.iloc[i*10:(i+1)*10, 2] = resultado_de_la_clasificacion.split(", ")
        df.to_csv('amlo_clasify_chatpgt2.csv', index=False)
        print("Fin de la asignacion")
    
def classify_df_enumerate():
    for i in range(5):
        resultado_de_la_clasificacion = clasificate_csv(result2[i])
        resultado_de_la_clasificacion = re.sub(r'([^A-Za-z\r\n\t])', '',resultado_de_la_clasificacion)
        print(resultado_de_la_clasificacion.split("\n"))
        print(len(resultado_de_la_clasificacion.split("\n")))
        print(type(resultado_de_la_clasificacion))
       
        df.iloc[i*10:(i+1)*10, 3] = resultado_de_la_clasificacion.split("\n")
        #df.to_csv('amlo_clasify_chatpgt2.csv', index=False)
        print("Fin de la asignacion")    
    

In [420]:
classify_df()

In [573]:
mapper = {
    'Support': "apoyo",
    '\nSupport': "apoyo",
    'Support.': "apoyo",
    'Health': "salud",
    'Security': "seguridad",
    'Security.': "seguridad",
    "History": "historia",
    "Economy": "economia",
    "Economy.": "economia",
    "Opposition": "competencia",
    "Opposition.": "competencia",
    "Opinion": "opinion",
    "Foreign": "exterior",
    "Foreign.": "exterior",
    "Construction": "construccion",
    "Construction.": "construccion",
    "Corruption": "corrupcion",
    "Corruption.": "corrupcion",
    "Migration": "exterior",
    
    
    "Oposición": "oposicion",
    "Vivienda": "construccion",
    "Seguridad": "seguridad",
    "Historia": "historia",
    "Apoyo": "apoyo",
    "Economia": "economia",
    "Economía": "economia",
    "Opinión": "opinion",
    "Exterior": "exterior",
    "Salud": "salud",
    "Construcción": "construccion",
    "Corrupción": "corrupcion",
    "Construcción.": "construccion",
    "\nOpinion": "opinion",
    "\nCorruption": "corrupcion",
    "Opinion.": "opinion",
    "Apoyo (Support)": "apoyo",
    "Maiceo (Corruption)": "corrupcion",
    "Privatización (Economy).": "economia",
    "Oppinion": "opinion",
    "Extranjero": "exterior",
    "Presupuesto (Economy)": "economia",
    "apoyo": "apoyo",
    "Apoyo.": "apoyo",
    "Soporte": "apoyo",
    
    "Sports": "opinion",
    "Comunication": "opinion",
    "Democracia": "seguridad",
    "Hesitation": "seguridad",
    "Arqueología": "historia",
    "Educación": "apoyo",
    "Cultura": "historia",
    "Tourism": "exterior",
    "Turismo": "exterior",
    "Migración": "exterior",
    "Migration": "exterior",
    "Migration.": "exterior",
    "Education": "apoyo",
    "Agriculture": "construccion",
    "Legal": "competencia",

}
df['classification_spanish'] = df['new_classification'].map(mapper)

In [574]:
#df["third_classification"] = ''
#df = df.drop('third_classification', axis=1)
#df = df.drop('classification_spanish', axis=1)

df.head(50)

,Texto,Clasificacion,new_classification,classification_spanish
0,Buenos días y feliz año. Deseamos que nos vay...,saludos,Support,apoyo
1,Vamos a iniciar la semana como lo hacemos siem...,NaN,Health,salud
2,También vamos a informar este primer día hábil...,apoyo,Support,apoyo
3,Y ahora se va a informar porque se está optand...,apoyo,Support,apoyo
4,En esta etapa se combinan los bancos particula...,apoyo,Support,apoyo
5,"También Ariadna Montiel, que es la secretaria ...",apoyo,Support,apoyo
6,Pero tenerlas terminadas no significa tenerlas...,apoyo,Support,apoyo
7,"También lo ocurrido ayer en Chihuahua, en Ciud...",seguridad,Security,seguridad
8,"De tomas maneras mañana, aun cuando no esté a...",NaN,Construction,construccion
9,"Bueno, ahora que estuvo con nosotros el gober...",seguridad,Security,seguridad


In [575]:
cuantos_nan = 0
for i in df.loc[pd.isna(df["classification_spanish"]), :].index:
    print(i)
    cuantos_nan +=1 
print(cuantos_nan)

747
1270
1959
2650
3075
3082
4509
4739
5306
5389
5509
5959
6264
7006
7145
8979
9620
9621
9622
9623
9625
9628
10149
11019
12444
13788
13864
15514
16626
17773
18440
19089
19595
19890
20030
21516
21760
21761
21762
21763
21764
41


In [577]:
df.iloc[13788]
#df["new_classification"].unique()
#datos atipicos 200 

Texto                     O sea, ¿por qué no ayudan? Vamos a decir, está...
Clasificacion                                                           NaN
new_classification                                                Oppostion
classification_spanish                                                  NaN
Name: 13788, dtype: object

## 1. Ahorita le mande 50 con el nuevo prompt y si los clasifico de a 10
- Opinion, Support, Support, Support, Support, Support, Support, Security, Support, Opposition
- Security, Support, Opinion, History, History, History, History, Opposition, Opposition, Opposition
- History, History, Opinion, Opinion, Economy, Economy, Support, Construction, Construction, Construction
- Economy, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition.
- Opinion, Opinion, Opposition, Opposition, Opposition, Opinion, Opinion, Opposition, Economy, Security.

### 2. Prompt llamado Bob
- Opinion, Health, Support, Support, Support, Construction, Construction, Security, Support, Security
- Security, History, Support, Opinion, History, History, History, Foreign, Opposition, Opposition
- History, History, Opinion, Opinion, Economy, Economy, Support, Construction, Construction, Construction
- Economy, Opposition, Opposition, Opposition, Corruption, Opposition, Opposition, Corruption, Opposition, Opinion

### 3. Prompt Bob mejorado (de mejor  no tiene nada)
13

- Opinion, Support, Support, Support, Support, Support, Support, Security, Construction, Support
- Seguridad, Apoyo, Apoyo
- Apoyo, Apoyo, Apoyo, Apoyo, Apoyo
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Opposición, Opposición
- History, Opinion, Opinion, Opinion, Economy, Economy, Construction, Construction, Construction, Construction
- Economy, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition
- Opinion, Opinion, Opposition, Opposition, Opposition, Opinion, Opinion, Opposition, Economy, Security


### 4. Prompt sin ser Bob pero mejor 

- Opinion, Support, Support, Support, Support, Construction, Construction, Security, Support, Security
- Seguridad, Apoyo, Seguridad, Historia, Historia, Historia, Historia, Historia, Historia, Historia
- History, Opinion, Opinion, Opinion, Economy, Economy, Support, Construction, Construction, Construction
- Economy, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition
- Opinion, Opinion, Opposition, Opposition, Opposition, Opinion, Opinion, Opposition, Economy, Security

### 5. Prompt anterior pero reiterando que no puede responder en Ingles
- Opinion, Support, Health, Support, Support, Support, Support, Security, Construction, Opposition.
- Seguridad, Apoyo, Seguridad, Historia, Historia, Historia, Historia, Historia, Historia, Historia
- History, Opposition, Opinion, Opinion, Economy, Economy, Construction, Construction, Construction, Construction

In [374]:
df.to_csv('amlo_clasify_chatpgt3.csv', index=False)

In [247]:
df.iloc[21759]

Texto                  Pues no podemos hacerlo, para eso están los o...
Clasificacion                                                       NaN
new_classification                                           Opposition
Name: 21759, dtype: object